# Politecnico Malvinas Argentinas- PDH: Preprocesamiento, Normalización y Técnicas Básicas

In [2]:
# Instalación de librerías necesarias (si es necesario en Colab)
!pip install nltk spacy scikit-learn thefuzz stop_words
!pip install fuzzywuzzy python-Levenshtein

import re
import nltk
import spacy
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

ModuleNotFoundError: No module named 'pandas'

In [3]:
# Descargar recursos de NLTK
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\celem\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\celem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# Cargar modelo en español de spaCy
!python -m spacy download es_core_news_md
nlp = spacy.load("es_core_news_md")

     ---------------------------------------- 0.0/42.3 MB ? eta -:--:--
     ---------------------------------------- 0.3/42.3 MB ? eta -:--:--
     - -------------------------------------- 1.8/42.3 MB 6.7 MB/s eta 0:00:07
     -- ------------------------------------- 3.1/42.3 MB 6.8 MB/s eta 0:00:06
     --- ------------------------------------ 3.9/42.3 MB 6.7 MB/s eta 0:00:06
     ---- ----------------------------------- 5.0/42.3 MB 5.6 MB/s eta 0:00:07
     ----- ---------------------------------- 6.0/42.3 MB 5.3 MB/s eta 0:00:07
     ------ --------------------------------- 7.1/42.3 MB 5.3 MB/s eta 0:00:07
     ------- -------------------------------- 8.4/42.3 MB 5.4 MB/s eta 0:00:07
     --------- ------------------------------ 9.7/42.3 MB 5.4 MB/s eta 0:00:06
     ---------- ----------------------------- 11.0/42.3 MB 5.6 MB/s eta 0:00:06
     ----------- ---------------------------- 12.6/42.3 MB 5.7 MB/s eta 0:00:06
     ------------- -------------------------- 13.9/42.3 MB 5.8 M

# 1. Preprocesamiento de texto

# Tokenizacion

Quizás una de las operaciones más básicas que podemos hacer es contar palabras. Imaginemos un caso extremadamente sencillo relacionado al aprendizaje automático. Pensemos en un clasificador de positividad / negatividad.

Podríamos usar una regla como:

si "bueno" está en el texto, clasificamos como positivo.

Claramente un enfoque tan sencillo va a ser propenso a muchos errores. Iremos viendo cómo aplicarlo y mejorarlo.

La operación de separar los textos en unidades básicas (o tokens) la llamamos tokenización.

In [5]:
doc2 = 'Usualmente, existe una relación costo-beneficio entre las distintas técnicas.'
doc2.split(' ')

['Usualmente,',
 'existe',
 'una',
 'relación',
 'costo-beneficio',
 'entre',
 'las',
 'distintas',
 'técnicas.']

En este codigo primero, se crea una variable llamada doc2 que contiene la cadena de texto: "Usualmente, existe una relación costo-beneficio entre las distintas técnicas."
Luego, se aplica el método split(' ') a esta cadena.

El propósito de split(' ') es dividir la cadena de texto en una lista de palabras, utilizando el espacio como separador. Este es un método fundamental para el procesamiento de texto que permite trabajar con palabras individuales.
Posibles mejoras o limitaciones:

El código es básico y funcional, pero no maneja puntuación. Las palabras con puntuación (como "técnicas.") se mantienen con el signo de puntuación.
No normaliza el texto (convertir a minúsculas, eliminar acentos, etc.).
Solo divide por espacios simples, lo que podría ser problemático si hay espacios múltiples o tabulaciones.

Vemos que en castellano, el lenguaje natural (escrito) puede tener distintas fuentes de variaciones:

Mayúsculas
Tildes
Signos ortográficos
Errores de tipeo
Variaciones propias del lenguaje como la conjugación
Artefactos de la escritura informal como "holaaa"
¿Cómo abordarán esto nuestros algorítmos?

Una solución es la normalización: con acepción similar aunque distinta a la que usamos en estadística, va a significar transformar a los strings a representaciones iguales.

Podemos:

Sacar espacios
Pasar a minúsculas
Quitar tildes

In [6]:
%%capture
!pip install unidecode

In [7]:
import unidecode
[unidecode.unidecode(w.lower().strip()) for w in doc2.split(' ')]

['usualmente,',
 'existe',
 'una',
 'relacion',
 'costo-beneficio',
 'entre',
 'las',
 'distintas',
 'tecnicas.']

la biblioteca unidecode, permite eliminar caracteres diacríticos (acentos, tildes, etc.)

Toma cada palabra w del resultado de doc2.split(' ')
Aplica w.lower() para convertir todo a minúsculas
Usa strip() para eliminar espacios en blanco al inicio y final de cada palabra
Aplica unidecode.unidecode() para eliminar acentos y otros caracteres especiales

El propósito es normalizar el texto para análisis, haciendo que palabras como "Técnicas" y "tecnicas" se consideren iguales.
Posibles mejoras o limitaciones:

Aún no elimina los signos de puntuación, por lo que "técnicas." se convertiría en "tecnicas."
Usa split(' ') que podría fallar con espacios múltiples o tabulaciones
No elimina palabras vacías (stopwords) como "entre", "las", etc., que suelen ser poco informativas

Otras normalizaciones más sofisticadas serían:
- Pasar de plural a singular
- Convertir el género de la palabra
- Conjugar los verbos en infinitivo

¡Es de esperarse que esto requiera métodos más complejos!

Vemos que con la tokenización anterior, todavía tenemos palabras con signos de puntuación pegados. Armar nuestro vocabulario es un paso clave para nuestros modelos de *Machine Learning*. 

In [12]:
## importar la función word_tokenize desde el módulo nltk.tokenize de la librería NLTK. 

In [13]:
import nltk
from nltk.tokenize import RegexpTokenizer

# Creamos un tokenizador personalizado
tokenizer = RegexpTokenizer(r'\w+')

text = "¡Hola! ¿Cómo estás? Estoy aprendiendo NLP con spaCy y NLTK. NLP es fascinante."

# Tokenización y Normalización
words = tokenizer.tokenize(text.lower())
print("Tokenización y Normalización:", words)

Tokenización y Normalización: ['hola', 'cómo', 'estás', 'estoy', 'aprendiendo', 'nlp', 'con', 'spacy', 'y', 'nltk', 'nlp', 'es', 'fascinante']


In [15]:
import nltk
from nltk.tokenize import word_tokenize

text = "¡Hola! ¿Cómo estás? Estoy aprendiendo NLP con spaCy y NLTK. NLP es fascinante."

# Tokenización y Normalización
words = word_tokenize(text.lower())
print("Tokenización y Normalización:", words)

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\celem/nltk_data'
    - 'C:\\Users\\celem\\anaconda3\\envs\\reportes\\nltk_data'
    - 'C:\\Users\\celem\\anaconda3\\envs\\reportes\\share\\nltk_data'
    - 'C:\\Users\\celem\\anaconda3\\envs\\reportes\\lib\\nltk_data'
    - 'C:\\Users\\celem\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


Importa nltk, que es la biblioteca Natural Language Toolkit para procesamiento de lenguaje natural.
Importa específicamente la función word_tokenize de nltk.tokenize.

Definición del texto:

Crea una variable text con una cadena de texto en español que incluye caracteres especiales, signos de puntuación y mayúsculas.

Tokenización y normalización:

Convierte todo el texto a minúsculas usando text.lower()
Aplica word_tokenize() para dividir el texto en tokens individuales (palabras, signos de puntuación, etc.)


Posibles mejoras o limitaciones:

Idioma: El tokenizador por defecto está optimizado para inglés. Para español, deberías especificar el idioma: word_tokenize(text.lower(), language="spanish").
Manejo de recursos: No incluye manejo de errores para cuando faltan recursos NLTK, lo que causa el error que mostraste.
Normalización limitada: Solo convierte a minúsculas, pero no elimina acentos ni realiza otras normalizaciones útiles para análisis de texto.

In [17]:

text = "¡Hola! ¿Cómo estás? Estoy aprendiendo NLP con spaCy y NLTK. NLP es fascinante."

# Tokenización y Normalización
words = word_tokenize(text.lower())
print("Tokenización y Normalización:", words)

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\celem/nltk_data'
    - 'C:\\Users\\celem\\anaconda3\\envs\\reportes\\nltk_data'
    - 'C:\\Users\\celem\\anaconda3\\envs\\reportes\\share\\nltk_data'
    - 'C:\\Users\\celem\\anaconda3\\envs\\reportes\\lib\\nltk_data'
    - 'C:\\Users\\celem\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


Definición del texto:

text = "¡Hola! ¿Cómo estás? Estoy aprendiendo NLP con spaCy y NLTK. NLP es fascinante."
Define una cadena de texto en español con diversos elementos (signos de puntuación, mayúsculas, acentos)

Tokenización y normalización:

text.lower(): Convierte todo el texto a minúsculas, eliminando la variabilidad por capitalización
word_tokenize(): Divide el texto en tokens individuales (palabras y signos de puntuación)
Este proceso es fundamental para cualquier tarea de NLP, ya que convierte texto continuo en unidades discretas para análisis

Visualización de resultados:

print("Tokenización y Normalización:", words): Muestra el resultado del procesamiento


Posibles mejoras o limitaciones:

Falta de especificación de idioma:

No especifica el idioma español, por lo que usa el modelo para inglés por defecto
Mejora: word_tokenize(text.lower(), language="spanish")


Normalización limitada:

Solo realiza conversión a minúsculas
Mejora: Incluir eliminación de acentos, caracteres especiales


Sin manejo de errores:

No tiene código para manejar errores de recursos faltantes de NLTK
Mejora: Agregar try/except o verificación de recursos


Procesamiento incompleto:

No incluye pasos adicionales como eliminación de stopwords o lematización
Mejora: Agregar estos procesamientos para análisis más completos


Tokens de puntuación:

El tokenizador separa la puntuación como tokens individuales
Si no se desea esto, se podría filtrar con una lista de comprensión

In [18]:
# Expresiones Regulares
text_clean = re.sub(r'[^a-zA-Záéíóúüñ ]', '', text.lower())
print("Texto limpio con regex:", text_clean)

Texto limpio con regex: hola cómo estás estoy aprendiendo nlp con spacy y nltk nlp es fascinante


Posibles mejoras o limitaciones

Falta de importación:

No se importa la biblioteca re (expresiones regulares)
Mejora: Agregar import re al inicio del código


Posible pérdida de información útil:

Elimina todos los signos de puntuación, lo que puede ser importante para análisis de sentimiento o estructura de oraciones
Elimina números, que podrían ser relevantes en algunos análisis


Manejo limitado de caracteres especiales:

La expresión podría no contemplar todos los caracteres especiales del español o de otros idiomas
Mejora: Ampliar la expresión regular para incluir más caracteres según necesidades


Agresividad de la limpieza:

Elimina puntuación que podría ser útil para identificar límites de oraciones
Mejora: Considerar un enfoque más selectivo para mantener algunos elementos de puntuación


Posible optimización:

La expresión regular podría ser más eficiente o más específica según el caso de uso

In [19]:
# Stop Words
stop_words = set(stopwords.words('spanish'))
filtered_words = [word for word in words if word not in stop_words]
print("Sin Stop Words:", filtered_words)

NameError: name 'stopwords' is not defined

Propósito: Esta línea tiene como objetivo crear un conjunto (set) de palabras consideradas "vacías".La librería nltk.corpus proporciona una lista de estas palabras comunes que generalmente no aportan significado relevante al analizar texto. Al convertir esta lista a un conjunto (set), se optimiza la búsqueda y verificación de pertenencia de palabras en etapas posteriores.
filtered_words = [word for word in words if word not in stop_words]:

Esta línea utiliza una construcción concisa conocida como "list comprehension" para crear una nueva lista llamada filtered_words. Itera a través de una lista preexistente llamada words .Para cada word en la lista words, se verifica si esa word no se encuentra dentro del conjunto de stop_words que se creó en el paso anterior. Si la palabra no es una "stop word", se añade a la nueva lista filtered_words. En esencia, esta línea filtra la lista original de palabras, removiendo aquellas que se consideran irrelevantes.
print("Sin Stop Words:", filtered_words):

Esta línea simplemente imprime en la consola una etiqueta ("Sin Stop Words:") seguida del contenido de la lista filtered_words. Esto permite visualizar el resultado del proceso de eliminación de las "stop words".
Posibles mejoras o limitaciones:

Sensibilidad a mayúsculas y minúsculas: El código tal como está es sensible a la capitalización. Por ejemplo, "El" y "el" se considerarían palabras diferentes.
Mejora: Antes de eliminar las "stop words", se podría convertir todas las palabras de la lista words a minúsculas (o mayúsculas) para asegurar una comparación uniforme.

In [23]:
# Stemming con NLTK
from nltk.corpus import stopwords

words = ["Este", "es", "un", "ejemplo", "de", "algunas", "palabras", "de", "prueba"] # Reemplaza con tus palabras originales
stop_words = set(stopwords.words('spanish'))
filtered_words = [word for word in words if word not in stop_words]
print("Sin Stop Words:", filtered_words)

Sin Stop Words: ['Este', 'ejemplo', 'palabras', 'prueba']


In [24]:
# Stemming con NLTK
stemmer = SnowballStemmer("spanish")
stemmed_words = [stemmer.stem(word) for word in filtered_words]
print("Stemming:", stemmed_words)

Stemming: ['este', 'ejempl', 'palabr', 'prueb']


Posibles mejoras o limitaciones:

Dependencia de filtered_words: Este código asume que la variable filtered_words ya está definida y contiene una lista de palabras que han sido previamente procesadas (por ejemplo, eliminando las "stop words"). Si filtered_words no existe o está vacía, el código no producirá el resultado esperado o podría generar un error.

Mejora: Sería más robusto incluir en el mismo script o asegurar que se ejecute previamente el código para obtener filtered_words.

Limitación: Al igual que la sobregeneralización, esta es una limitación propia de los algoritmos de stemming basados en reglas.
Necesidad de importar la librería: Como vimos en la conversación anterior, para que SnowballStemmer funcione, es necesario haber importado previamente la clase desde la librería nltk.stem.

Mejora: Asegurarse de incluir la línea from nltk.stem import SnowballStemmer al principio del script para evitar errores.
Idioma específico: El stemmer se ha inicializado específicamente para el idioma español. Si se quisiera aplicar stemming a texto en otro idioma, se necesitaría crear una instancia de SnowballStemmer con el parámetro del idioma correspondiente (e.g., "english", "french", etc.).

Limitación: El código tal cual está solo funciona para el español.

In [25]:
# Lemmatización con spaCy
doc = nlp(" ".join(filtered_words))
lemmas = [token.lemma_ for token in doc]
print("Lemmatización:", lemmas)

Lemmatización: ['este', 'ejemplo', 'palabra', 'probar']


In [27]:
# Distancias entre strings
# Distancias entre strings
from fuzzywuzzy import fuzz


Distancia fuzzy entre 'casa' y 'casas': 89


Propósito de cada etapa:

doc = nlp(" ".join(filtered_words)):

Propósito: Esta línea toma la lista de palabras filtradas (filtered_words) y las une en una única cadena de texto, separadas por espacios (" "). Luego, pasa esta cadena al objeto nlp. Se asume que nlp es un objeto de lenguaje de spaCy que ha sido previamente cargado con un modelo específico para el idioma español (por ejemplo, nlp = spacy.load("es_core_news_sm")). Al procesar la cadena con nlp, spaCy realiza un análisis lingüístico completo del texto, tokenizando las palabras y realizando tareas como el etiquetado gramatical (part-of-speech tagging), la identificación de entidades nombradas y, crucialmente para este caso, la determinación del lema de cada token. El resultado de este procesamiento es un objeto Doc que contiene toda la información lingüística analizada.
lemmas = [token.lemma_ for token in doc]:

Propósito: Esta línea utiliza una "list comprehension" para extraer el lema de cada token dentro del objeto Doc que se generó en el paso anterior. El objeto Doc en spaCy contiene una secuencia de objetos Token, cada uno representando una palabra en el texto procesado. Cada objeto Token tiene varios atributos, y .lemma_ es uno de ellos que contiene la forma base o lema de la palabra. Esta línea itera sobre cada token en el doc y extrae su lema, almacenándolos en una nueva lista llamada lemmas.
print("Lemmatización:", lemmas):

Propósito: Esta línea simplemente imprime en la consola una etiqueta ("Lemmatización:") seguida del contenido de la lista lemmas. Esto permite visualizar las formas base de las palabras originales después del proceso de lematización realizado por spaCy.
Posibles mejoras o limitaciones:

Dependencia de filtered_words y nlp: Al igual que con el stemming, este código depende de la existencia previa de la variable filtered_words (la lista de palabras sin "stop words"). Además, crucialmente, depende de que el objeto de lenguaje de spaCy (nlp) haya sido cargado correctamente con un modelo para el idioma español. Si nlp no está definido o no se ha cargado el modelo adecuado, el código generará un error.

Mejora: Asegurarse de que el modelo de spaCy para español esté instalado (pip install spacy && python -m spacy download es_core_news_sm) y que el objeto nlp se haya cargado correctamente antes de ejecutar este código.
Precisión de la lematización: La precisión de la lematización depende en gran medida de la calidad del modelo de lenguaje de spaCy que se esté utilizando. Los modelos más grandes suelen ser más precisos pero también más lentos y requieren más recursos.

Requisitos de la librería spaCy: Este código requiere que la librería spaCy esté instalada en el entorno de Python. Si no lo está, se producirá un error al intentar ejecutar el código.

Mejora: Incluir una verificación o instrucción para instalar spaCy si no está presente.

Propósito de cada etapa:

from fuzzywuzzy import fuzz:

Esta línea importa el módulo fuzz desde la librería fuzzywuzzy. La librería fuzzywuzzy se utiliza para calcular la similitud entre secuencias de texto, como strings. El módulo fuzz contiene varias funciones que implementan diferentes algoritmos para medir esta similitud, incluyendo la función ratio que se utiliza en la siguiente línea.
print("Distancia fuzzy entre 'casa' y 'casas':", fuzz.ratio("casa", "casas")):

Esta línea utiliza la función fuzz.ratio() para calcular la similitud entre las dos cadenas de texto proporcionadas: "casa" y "casas". La función fuzz.ratio() devuelve un puntaje entre 0 y 100, donde 100 indica una coincidencia perfecta. En este caso, calculará qué tan similares son las dos palabras basándose en la distancia de Levenshtein (la cantidad mínima de ediciones de un solo carácter requeridas para cambiar una palabra en la otra) y otros factores. El resultado de esta comparación se imprime en la consola junto con una etiqueta descriptiva.
texts = ["El gato come pescado", "El perro ladra en la casa"]:

Propósito: Esta línea simplemente define una lista llamada texts que contiene dos cadenas de texto como elementos. Aunque esta lista se define en el código, no se utiliza directamente en las líneas anteriores para calcular distancias fuzzy. Probablemente, esta línea se incluye como un ejemplo o como preparación para realizar comparaciones entre estas frases en un código posterior (que no se muestra en este fragmento).
Posibles mejoras o limitaciones:

Limitación del fuzz.ratio(): La función fuzz.ratio() es útil para medir la similitud general entre dos cadenas. Sin embargo, puede no ser óptima en todos los casos. Por ejemplo, podría no manejar de manera ideal las diferencias en el orden de las palabras en frases largas.

Mejora: La librería fuzzywuzzy ofrece otras funciones más especializadas como fuzz.partial_ratio(), fuzz.token_sort_ratio(), y fuzz.token_set_ratio() que pueden ser más adecuadas para diferentes tipos de comparaciones (por ejemplo, cuando una cadena es una subcadena de otra o cuando el orden de las palabras varía). Considerar el uso de estas funciones dependiendo del caso de uso.
Sensibilidad a errores tipográficos: Si bien fuzzywuzzy está diseñado para manejar errores tipográficos, la efectividad puede variar dependiendo de la cantidad y el tipo de errores.

Consideración: Para casos con muchos errores tipográficos o variaciones fonéticas, se podrían explorar otras librerías o técnicas más avanzadas.
Rendimiento para grandes conjuntos de datos: Si necesitas comparar una gran cantidad de strings entre sí, calcular la distancia fuzzy pairwise puede ser computacionalmente costoso.

Mejora: Para grandes conjuntos de datos, se podrían considerar técnicas de indexación o algoritmos más eficientes para la búsqueda de similitudes aproximadas.
Aplicación a nivel de palabras vs. frases: El ejemplo principal compara palabras individuales. Para comparar la similitud entre frases, fuzz.ratio() funciona, pero las alternativas mencionadas (partial_ratio, token_sort_ratio, token_set_ratio) podrían ofrecer resultados más intuitivos en ciertos escenarios donde el orden de las palabras o la presencia de palabras adicionales varía.

Necesidad de instalación: Para utilizar fuzzywuzzy, la librería debe estar instalada en el entorno de Python.

Consideración: Al compartir código que utiliza fuzzywuzzy, es importante mencionar la necesidad de instalar la librería (pip install fuzzywuzzy).

In [28]:
# segundo ejemplo %%capture
!pip install pyjarowinkler
from nltk.metrics import edit_distance
from pyjarowinkler import distance as jwdist

In [29]:
# Definimos una lista de tuplas de palabras
palabras = [("pero", "perro"),("pero", "pierdo"), ("nueve", "mueve"),  ("totalmente","diferentes"), ("pero", "basta")]

# Calculamos las metricas de distancia pasando cada tupla como argumentos a levdist() y get_jaro_distance()
for x,y in palabras:
    print(f"'{x}' vs '{y}':")
    print("Distancia Levenshtein ->", edit_distance(x,y))
    print("Similitud Jaro Winkler ->",jwdist.get_jaro_distance(x,y))
    print("-"*40)

'pero' vs 'perro':
Distancia Levenshtein -> 1
Similitud Jaro Winkler -> 0.07
----------------------------------------
'pero' vs 'pierdo':
Distancia Levenshtein -> 2
Similitud Jaro Winkler -> 0.11
----------------------------------------
'nueve' vs 'mueve':
Distancia Levenshtein -> 1
Similitud Jaro Winkler -> 0.13
----------------------------------------
'totalmente' vs 'diferentes':
Distancia Levenshtein -> 7
Similitud Jaro Winkler -> 0.48
----------------------------------------
'pero' vs 'basta':
Distancia Levenshtein -> 5
Similitud Jaro Winkler -> 1.0
----------------------------------------


¡Entendido! Ahora vamos a analizar este código que calcula las distancias entre pares de palabras utilizando las métricas de Levenshtein y Jaro-Winkler. Aquí te explico el propósito de cada etapa y algunas posibles mejoras o limitaciones:

Propósito de cada etapa:

palabras = [("pero", "perro"),("pero", "pierdo"), ("nueve", "mueve"), ("totalmente","diferentes"), ("pero", "basta")]:

Propósito: Esta línea define una lista llamada palabras. Cada elemento de esta lista es una tupla que contiene dos strings (pares de palabras). Estos pares de palabras serán utilizados como entrada para calcular las métricas de distancia en el siguiente bucle.
for x,y in palabras::

Propósito: Esta línea inicia un bucle for que itera a través de cada tupla en la lista palabras. En cada iteración, los dos elementos de la tupla se desempaquetan y se asignan a las variables x e y, respectivamente. Estas variables representarán las dos palabras que se compararán en cada paso del bucle.
print(f"'{x}' vs '{y}':"):

Propósito: Dentro del bucle, esta línea imprime en la consola las dos palabras que se están comparando en la iteración actual, encerradas entre comillas simples y precedidas por una etiqueta descriptiva. Esto ayuda a identificar qué par de palabras se está analizando en cada salida.
print("Distancia Levenshtein ->", edit_distance(x,y)):

Propósito: Esta línea calcula la distancia de Levenshtein entre las palabras x e y utilizando la función edit_distance(). La distancia de Levenshtein mide el número mínimo de ediciones de un solo carácter (inserciones, eliminaciones o sustituciones) requeridas para cambiar una palabra en la otra. El resultado de este cálculo se imprime en la consola junto con una etiqueta indicando que se trata de la distancia de Levenshtein. Es importante notar que para que edit_distance() funcione, esta función debe estar definida o importada previamente desde una librería como nltk.metrics o Levenshtein.
print("Similitud Jaro Winkler ->",jwdist.get_jaro_distance(x,y)):

Propósito: Esta línea calcula la similitud Jaro-Winkler entre las palabras x e y utilizando la función get_jaro_distance() del módulo jwdist. La similitud Jaro-Winkler es una métrica que mide la similitud entre dos secuencias de caracteres. Tiene en cuenta la cantidad de caracteres coincidentes, las transposiciones y da más peso a las coincidencias al principio de las cadenas. El resultado es un valor entre 0 (sin similitud) y 1 (coincidencia perfecta). Este valor se imprime en la consola junto con una etiqueta indicando que se trata de la similitud Jaro-Winkler. De manera similar a edit_distance(), para que jwdist.get_jaro_distance() funcione, el módulo jwdist debe estar definido o importado previamente, probablemente desde una librería como jellyfish.
print("-"*40):

Propósito: Finalmente, esta línea imprime una línea horizontal de 40 guiones (-) en la consola. Esto se utiliza como separador visual entre los resultados de la comparación de cada par de palabras, lo que facilita la lectura de la salida.
Posibles mejoras o limitaciones:

Dependencia de librerías no especificadas: El código utiliza las funciones edit_distance() y jwdist.get_jaro_distance(), pero no se especifica de dónde provienen. Para que este código sea ejecutable, es necesario importar las librerías correspondientes.

Mejora: Al principio del script, se deberían incluir las importaciones necesarias:

# 3. Vectorización y Representación del Texto

In [30]:
texts = ["El gato come pescado", "El perro ladra en la casa"]

In [36]:
pip install pandas


   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.5 MB 4.2 MB/s eta 0:00:03
   ------ --------------------------------- 1.8/11.5 MB 5.9 MB/s eta 0:00:02
   ------ --------------------------------- 1.8/11.5 MB 5.9 MB/s eta 0:00:02
   ------------- -------------------------- 3.9/11.5 MB 5.5 MB/s eta 0:00:02
   ----------------- ---------------------- 5.0/11.5 MB 5.5 MB/s eta 0:00:02
   ----------------------- ---------------- 6.8/11.5 MB 6.0 MB/s eta 0:00:01
   -------------------------- ------------- 7.6/11.5 MB 5.7 MB/s eta 0:00:01
   ------------------------------ --------- 8.7/11.5 MB 6.0 MB/s eta 0:00:01
   ------------------------------ --------- 8.7/11.5 MB 6.0 MB/s eta 0:00:01
   ------------------------------ --------- 8.7/11.5 MB 6.0 MB/s eta 0:00:01
   ------------------------------- -------- 9.2/11.5 MB 4.3 MB/s eta 0:00:01
   ----------------------------------- ---- 10.2/11.5 MB 4.3 MB/s eta 0:00:01
   --

In [37]:
# Bolsa de Palabras (BoW)
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)
pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())


casa  come  el  en  gato  la  ladra  perro  pescado
0     0     1   1   0     1   0      0      0        1
1     1     0   1   1     0   1      1      1        0

Propósito de cada etapa:

from sklearn.feature_extraction.text import CountVectorizer: Importa la clase CountVectorizer desde la librería scikit-learn, que se utiliza para convertir texto en una matriz de conteo de palabras.
import pandas as pd: Importa la librería pandas, que se utiliza para la manipulación y análisis de datos, y le asigna el alias pd para facilitar su uso.
vectorizer = CountVectorizer(): Crea una instancia del objeto CountVectorizer. Este objeto se utilizará para analizar y transformar los textos.
X = vectorizer.fit_transform(texts):
fit(texts): El vectorizer analiza la lista de textos (texts) para aprender el vocabulario único presente en ellos.
transform(texts): Utiliza el vocabulario aprendido para convertir cada texto en un vector de conteo, donde cada posición corresponde a una palabra del vocabulario y el valor es la frecuencia de esa palabra en el texto. El resultado es una matriz dispersa X.
pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out()):
X.toarray(): Convierte la matriz dispersa X en un array NumPy denso.
columns=vectorizer.get_feature_names_out(): Obtiene la lista de palabras (características) del vocabulario aprendido por el vectorizer para usarlas como nombres de las columnas del DataFrame.
pd.DataFrame(...): Crea un DataFrame de pandas a partir del array denso y los nombres de las columnas, lo que proporciona una representación tabular de la Bolsa de Palabras.
Posibles mejoras o limitaciones (como discutimos antes):

Sensibilidad a la capitalización y puntuación: Por defecto, se ignoran. Se pueden ajustar los parámetros de CountVectorizer.
No considera el significado ni el orden de las palabras: Es una representación simple basada en la frecuencia. Para análisis más profundos, se requieren técnicas más avanzadas.
Problema de dimensionalidad: Vocabularios grandes pueden llevar a matrices dispersas de alta dimensión. Se pueden usar parámetros como max_features, min_df, max_df para controlar esto.
No pondera la importancia de las palabras: Todas las palabras se tratan por igual. TF-IDF es una alternativa que sí pondera.
Necesidad de preprocesamiento: La eliminación de "stop words", la lematización o el stemming pueden mejorar la calidad de la representación BoW.

In [39]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

vectorizer_tfidf = TfidfVectorizer()
X_tfidf = vectorizer_tfidf.fit_transform(texts)
pd.DataFrame(X_tfidf.toarray(), columns=vectorizer_tfidf.get_feature_names_out())

casa      come        el       en      gato       la    ladra    perro  \
0  0.00000  0.534046  0.379978  0.00000  0.534046  0.00000  0.00000  0.00000   
1  0.42616  0.000000  0.303216  0.42616  0.000000  0.42616  0.42616  0.42616   

    pescado  
0  0.534046  
1  0.000000

Propósito de cada etapa:

from sklearn.feature_extraction.text import TfidfVectorizer: Importa la clase TfidfVectorizer desde la librería scikit-learn. Esta clase se utiliza para convertir una colección de documentos de texto en una matriz de características TF-IDF.
import pandas as pd: Importa la librería pandas, asignándole el alias pd. Pandas se utiliza para crear y manipular DataFrames, que son estructuras de datos tabulares útiles para visualizar y trabajar con los resultados.
vectorizer_tfidf = TfidfVectorizer(): Crea una instancia del objeto TfidfVectorizer. Este objeto aprenderá los pesos TF-IDF de las palabras en el corpus y transformará los textos en vectores TF-IDF.
X_tfidf = vectorizer_tfidf.fit_transform(texts):
fit(texts): El vectorizer_tfidf analiza la lista de textos (texts) para aprender el vocabulario y calcular las estadísticas de frecuencia de los términos (document frequency).
transform(texts): Utiliza el vocabulario y las estadísticas aprendidas para transformar cada texto en un vector TF-IDF. El valor de cada dimensión en este vector representa el peso TF-IDF de la palabra correspondiente en ese documento. El resultado X_tfidf es una matriz dispersa donde la mayoría de los valores pueden ser cero.
pd.DataFrame(X_tfidf.toarray(), columns=vectorizer_tfidf.get_feature_names_out()):
X_tfidf.toarray(): Convierte la matriz dispersa X_tfidf en un array NumPy denso.
columns=vectorizer_tfidf.get_feature_names_out(): Obtiene la lista de palabras (características) del vocabulario aprendido por el vectorizer_tfidf y las utiliza como los nombres de las columnas del DataFrame.
pd.DataFrame(...): Crea un DataFrame de pandas a partir del array denso y los nombres de las columnas, proporcionando una representación tabular de los pesos TF-IDF para cada palabra en cada texto.

In [40]:
# N-gramas
vectorizer_ngram = CountVectorizer(ngram_range=(2,3), min_df=1)
X_ngram = vectorizer_ngram.fit_transform(texts)
pd.DataFrame(X_ngram.toarray(), columns=vectorizer_ngram.get_feature_names_out())


come pescado  el gato  el gato come  el perro  el perro ladra  en la  \
0             1        1             1         0               0      0   
1             0        0             0         1               1      1   

   en la casa  gato come  gato come pescado  la casa  ladra en  ladra en la  \
0           0          1                  1        0         0            0   
1           1          0                  0        1         1            1   

   perro ladra  perro ladra en  
0            0               0  
1            1               1

Propósito de cada etapa:

vectorizer_ngram = CountVectorizer(ngram_range=(2,3), min_df=1):

Propósito: Se crea una instancia de CountVectorizer con parámetros específicos para la extracción de N-gramas:
ngram_range=(2,3): Este parámetro le indica al CountVectorizer que genere tanto bigramas (secuencias de 2 palabras) como trigramas (secuencias de 3 palabras). Por ejemplo, en la frase "el gato come pescado", algunos bigramas serían "el gato", "gato come", "come pescado", y un trigrama sería "el gato come".
min_df=1: Este parámetro establece un umbral mínimo para la frecuencia del documento de los N-gramas. Un valor de 1 significa que un N-grama debe aparecer en al menos un documento para ser incluido en el vocabulario.
X_ngram = vectorizer_ngram.fit_transform(texts):

fit(texts): El vectorizer_ngram analiza la lista de textos (texts) para identificar todos los bigramas y trigramas que cumplen con el criterio de min_df=1 y construye un vocabulario de estos N-gramas.
transform(texts): Utilizando el vocabulario de N-gramas aprendido, este método convierte cada texto en un vector de conteo. Cada dimensión del vector corresponde a un N-grama del vocabulario, y el valor es la frecuencia de ese N-grama en el texto. El resultado X_ngram es una matriz dispersa que representa los conteos de los N-gramas en cada documento.
pd.DataFrame(X_ngram.toarray(), columns=vectorizer_ngram.get_feature_names_out()):

X_ngram.toarray(): Convierte la matriz dispersa X_ngram en un array NumPy denso.
columns=vectorizer_ngram.get_feature_names_out(): Obtiene la lista de N-gramas (bigramas y trigramas) que forman el vocabulario aprendido por el vectorizer_ngram y los utiliza como los nombres de las columnas del DataFrame.
pd.DataFrame(...): Crea un DataFrame de pandas a partir del array denso y los nombres de las columnas, proporcionando una representación tabular de la frecuencia de cada bigrama y trigrama en cada texto.
Si ejecutas este código, el DataFrame resultante mostrará las combinaciones de dos y tres palabras que aparecen en tus textos, junto con la frecuencia de cada una en cada texto. Esto puede ser útil para capturar información sobre la secuencia de palabras y el contexto local, algo que la simple Bolsa de Palabras no hace.

# 4. Tareas Avanzadas con spaCy

In [41]:
sentence = "El presidente de Argentina visitó Buenos Aires."
doc = nlp(sentence)

In [42]:
# POS Tagging
print([(token.text, token.pos_) for token in doc])

[('El', 'DET'), ('presidente', 'NOUN'), ('de', 'ADP'), ('Argentina', 'PROPN'), ('visitó', 'VERB'), ('Buenos', 'PROPN'), ('Aires', 'PROPN'), ('.', 'PUNCT')]


In [43]:
# Named Entity Recognition (NER)
print([(ent.text, ent.label_) for ent in doc.ents])

[('Argentina', 'LOC'), ('Buenos Aires', 'LOC')]


In [71]:
import spacy
from spacy import displacy

# Asegúrate de que el modelo de spaCy para español esté cargado
nlp = spacy.load("es_core_news_sm")

doc = nlp("El gato come pescado.") # Ejemplo de un documento

displacy.render(doc, style='dep', jupyter=True)

In [46]:

# Limitaciones de Bolsa de Palabras
print("BoW no captura el significado de las palabras ni el orden de las mismas.")

BoW no captura el significado de las palabras ni el orden de las mismas.


 la representación BoW es bastante básica y tiene las siguientes limitaciones inherentes:

No captura el significado de las palabras: BoW trata cada palabra como un token independiente, sin tener en cuenta su significado semántico. Palabras con significados similares (sinónimos) se consideran tokens diferentes, y palabras con múltiples significados (polisemia) se tratan de la misma manera en todos sus usos.

No captura el orden de las palabras: BoW ignora completamente la secuencia en la que aparecen las palabras en un documento. Frases con las mismas palabras pero en diferente orden tendrán la misma representación BoW, aunque su significado pueda ser completamente distinto. Por ejemplo, "el perro muerde al hombre" y "el hombre muerde al perro" tendrían la misma representación BoW.

#Sustituye el conjunto de datos original por otro (propio, de fuentes abiertas 
o generado artificialmente).

In [47]:
doc3 = 'Probando con otro ejemplo, utilizando técnicas aprendidas en clases de procesamiento del habla'
doc3.split(' ')

['Probando',
 'con',
 'otro',
 'ejemplo,',
 'utilizando',
 'técnicas',
 'aprendidas',
 'en',
 'clases',
 'de',
 'procesamiento',
 'del',
 'habla']

In [48]:
import unidecode
[unidecode.unidecode(w.lower().strip()) for w in doc3.split(' ')]

['probando',
 'con',
 'otro',
 'ejemplo,',
 'utilizando',
 'tecnicas',
 'aprendidas',
 'en',
 'clases',
 'de',
 'procesamiento',
 'del',
 'habla']

In [51]:
# Creamos un tokenizador personalizado
tokenizer = RegexpTokenizer(r'\w+')

text1 = "Probando con otro ejemplo, utilizando técnicas aprendidas en clases de procesamiento del habla"

# Tokenización y Normalización
words = tokenizer.tokenize(text1.lower())
print("Tokenización y Normalización:", words)

Tokenización y Normalización: ['probando', 'con', 'otro', 'ejemplo', 'utilizando', 'técnicas', 'aprendidas', 'en', 'clases', 'de', 'procesamiento', 'del', 'habla']


In [52]:
# Expresiones Regulares
text1_clean = re.sub(r'[^a-zA-Záéíóúüñ ]', '', text1.lower())
print("Texto limpio con regex:", text_clean)

Texto limpio con regex: hola cómo estás estoy aprendiendo nlp con spacy y nltk nlp es fascinante


In [54]:
# Stemming con NLTK
from nltk.corpus import stopwords

words = ['probando', 'con', 'otro', 'ejemplo', 'utilizando', 'técnicas', 'aprendidas', 'en', 'clases', 'de', 'procesamiento', 'del', 'habla']
stop_words = set(stopwords.words('spanish'))
filtered_words = [word for word in words if word not in stop_words]
print("Sin Stop Words:", filtered_words)

Sin Stop Words: ['probando', 'ejemplo', 'utilizando', 'técnicas', 'aprendidas', 'clases', 'procesamiento', 'habla']


In [55]:
# Stemming con NLTK
stemmer = SnowballStemmer("spanish")
stemmed_words = [stemmer.stem(word) for word in filtered_words]
print("Stemming:", stemmed_words)

Stemming: ['prob', 'ejempl', 'utiliz', 'tecnic', 'aprend', 'clas', 'proces', 'habl']


In [56]:
# Lemmatización con spaCy
doc = nlp(" ".join(filtered_words))
lemmas = [token.lemma_ for token in doc]
print("Lemmatización:", lemmas)

Lemmatización: ['probar', 'ejemplo', 'utilizar', 'técnica', 'aprendido', 'clase', 'procesamiento', 'hablar']


In [57]:
# 3. Vectorización y Representación del Texto

In [58]:
texts = ["El gato es un animal mas independiente", "El perro es un animal mas sociable"]

In [59]:
# Bolsa de Palabras (BoW)
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)
pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

animal  el  es  gato  independiente  mas  perro  sociable  un
0       1   1   1     1              1    1      0         0   1
1       1   1   1     0              0    1      1         1   1

In [60]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

vectorizer_tfidf = TfidfVectorizer()
X_tfidf = vectorizer_tfidf.fit_transform(texts)
pd.DataFrame(X_tfidf.toarray(), columns=vectorizer_tfidf.get_feature_names_out())

animal        el        es      gato  independiente       mas     perro  \
0  0.334251  0.334251  0.334251  0.469778       0.469778  0.334251  0.000000   
1  0.334251  0.334251  0.334251  0.000000       0.000000  0.334251  0.469778   

   sociable        un  
0  0.000000  0.334251  
1  0.469778  0.334251

In [61]:
# N-gramas
vectorizer_ngram = CountVectorizer(ngram_range=(2,3), min_df=1)
X_ngram = vectorizer_ngram.fit_transform(texts)
pd.DataFrame(X_ngram.toarray(), columns=vectorizer_ngram.get_feature_names_out())

animal mas  animal mas independiente  animal mas sociable  el gato  \
0           1                         1                    0        1   
1           1                         0                    1        0   

   el gato es  el perro  el perro es  es un  es un animal  gato es  \
0           1         0            0      1             1        1   
1           0         1            1      1             1        0   

   gato es un  mas independiente  mas sociable  perro es  perro es un  \
0           1                  1             0         0            0   
1           0                  0             1         1            1   

   un animal  un animal mas  
0          1              1  
1          1              1

# 4. Tareas Avanzadas con spaCy

In [62]:
sentence = "Ejemplo de prueba aprendiendo NPL."
doc = nlp(sentence)

In [63]:
# POS Tagging
print([(token.text, token.pos_) for token in doc])

[('Ejemplo', 'NOUN'), ('de', 'ADP'), ('prueba', 'NOUN'), ('aprendiendo', 'VERB'), ('NPL', 'PROPN'), ('.', 'PUNCT')]


In [64]:
# Named Entity Recognition (NER)
print([(ent.text, ent.label_) for ent in doc.ents])

[('NPL', 'MISC')]


In [69]:
!python -m spacy download es_core_news_sm


  Using cached https://github.com/explosion/spacy-models/releases/download/es_core_news_sm-3.8.0/es_core_news_sm-3.8.0-py3-none-any.whl (12.9 MB)
[+] Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [72]:
import spacy
from spacy import displacy

# Asegúrate de que el modelo de spaCy para español esté cargado
nlp = spacy.load("es_core_news_sm")

doc = nlp("El perro come alimento y se duerme.") # Ejemplo de un documento

displacy.render(doc, style='dep', jupyter=True)